**Business Overview:**

ExpressWay Logistics is a dynamic logistics service provider, committed to delivering efficient, reliable and cost-effective courier transportation and warehousing solutions. With a focus on speed, precision and customer satisfaction, we aim to be the go-to partner for our customers seeking seamless courier services. Our core service involves ensuring operational efficiency throughout our delivery and courier services, including inventory management, durable packaging and swift dispatch of couriers, real time tracking of shipments and on-time delivery of couriers as promised. We are committed to enhance our logistics and courier services and improve seamless connectivity for our customers.

**Objective:**

Our primary objective is to conduct a sentiment analysis of user-generated reviews across various digital channels and platforms. By paying attention to their feedback, we want to find ways to make our services better - like handling different customer demands simultaneously, dealing with late deliveries, and keeping packages secured and intact. Through the application of prompt engineering methodologies and sentiment analysis, we'll figure out if sentiments expressed by users for our courier services are Positive or Negative. This will help us understand where we need to improve in order to meet customer expectations and keep them happy. With a focus on getting better all the time, we'll overcome the challenges at ExpressWay Logistics and make our services the best.

**Current Challenge:**

ExpressWay Logistics faces numerous challenges in ensuring seamless deliveries and customer satisfaction. These challenges include managing various customer demands simultaneously, addressing delays in deliveries and ensuring products arrive intact and safe. Additionally, the company struggles with complexity of efficiently storing and handling a large volume of packages and ultimately meeting customer expectations. Moreover, maintaining a skilled workforce capable of handling various aspects of logistics operations presents its own set of challenges. Overcoming these obstacles requires a comprehensive approach that integrates innovative technology, strategic planning, and continuous improvement initiatives to ensure smooth operations and exceptional service delivery.

**Data Description:**

The dataset titled "courier-service_reviews.csv" is structured to facilitate sentiment analysis for courier service reviews. Here's a brief description of the data columns:

1. id: This column contains unique identifiers for each review entry. It helps in distinguishing and referencing individual reviews.
2. review: This column includes the actual text of the courier service reviews. The reviews are likely composed of customer opinions and experiences regarding different aspects of the services provided by ExpressWay Logistics.
3. sentiment: This column provides an additional layer of classification (positive and negative) for the mentioned reviews.

##**Step 1. Setup (2 Marks)**

(A) Writing/Creating the config.json file  ()

In [ ]:
!pip install tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.6 MB/s eta 0:00:00


In [ ]:
# Install the required packages
!pip install openai pandas scikit-learn tqdm tabulate

# Import necessary libraries
from openai import AzureOpenAI
import json
import random
import tiktoken
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tabulate import tabulate




In [ ]:
!pip install session-info


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 3.5 MB/s eta 0:00:00
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8023 sha256=44f04b29a5405a9075d166bb3a97b05f67a1411237987e062e8844d25d57b67a
  Stored in directory: /root/.cache/pip/wheels/6a/aa/b9/eb5d4031476ec10802795b97ccf937b9bd998d68a9b268765a
Successfully built session-info


In [ ]:
import session_info


In [ ]:
session_info.show()


### Authentication

**(A) Writing/Creating the config.json file ()**

In [ ]:
import json  # Import the JSON module
# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "",            #Replace it with your credentials
    "AZURE_OPENAI_ENDPOINT": "",      #Replace it with your credentials
    "AZURE_OPENAI_APIVERSION": "2024-05-01", #Replace it with your credentials
    "CHATGPT_MODEL": "gpt-35-turbo"             #Replace it with your credentials
}


# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")


Config file created successfully!


In [ ]:
with open('config.json', 'r') as az_creds:
    data = az_creds.read()

In [ ]:
creds = json.loads(data)

In [ ]:
def num_tokens_from_messages(messages):

    """
    Return the number of tokens used by a list of messages.
    Adapted from the Open AI cookbook token counter
    """

    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

    # Each message is sandwiched with <|start|>role and <|end|>
    # Hence, messages look like: <|start|>system or user or assistant{message}<|end|>

    tokens_per_message = 3 # token1:<|start|>, token2:system(or user or assistant), token3:<|end|>

    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))

    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>

    return num_tokens

In [ ]:
## Task : Sentiment Analysis

##**Step 2: Assemble Data (5 Marks)**

(A) Upload and Read csv File (2 Marks)

(B) Count Positive and Negative Sentiment Reviews (1 Marks)

(C) Split the Dataset (2 Marks)

In [ ]:
import pandas as pd



In [ ]:
from google.colab import files
import pandas as pd  # Import pandas

# Upload the CSV file
uploaded = files.upload()

# Load the dataset into a DataFrame
cs_reviews_df = "courier-service_reviews.csv"  # Replace with your file name
data = pd.read_csv(cs_reviews_df)

# Display the first few rows of the data to verify
data.head()


Saving courier-service_reviews.csv to courier-service_reviews (2).csv


,id,review,sentiment
0,1,ExpressWay Logistics' commitment to transparen...,Positive
1,2,The tracking system implemented by ExpressWay ...,Positive
2,3,ExpressWay Logistics is a lifesaver when it co...,Positive
3,4,Expressway Logistics is the worst courier serv...,Negative
4,5,ExpressWay Logistics failed to meet my expecta...,Negative


In [ ]:
# Display basic information about the dataset
print(data.info())
print(data.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         131 non-null    int64 
 1   review     131 non-null    object
 2   sentiment  131 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.2+ KB
None
   id                                             review sentiment
0   1  ExpressWay Logistics' commitment to transparen...  Positive
1   2  The tracking system implemented by ExpressWay ...  Positive
2   3  ExpressWay Logistics is a lifesaver when it co...  Positive
3   4  Expressway Logistics is the worst courier serv...  Negative
4   5  ExpressWay Logistics failed to meet my expecta...  Negative


In [ ]:
data.shape

(131, 3)

In [ ]:
print(data.sample(5))

      id                                             review sentiment
90    91  ExpressWay Logistics' proactive approach to pr...  Positive
109  110  As a frequent traveller, I often rely on shipp...  Positive
84    85  As a busy professional, I rely on efficient co...  Positive
22    23  The customer support team at ExpressWay Logist...  Positive
65    66  The lack of professionalism and accountability...  Negative


**(B) Count Positive and Negative Sentiment Reviews (1 Marks)**

In [ ]:
# Count positive and negative sentiments
sentiment_counts = Counter(data['sentiment'])
print("Sentiment Distribution:")
print(tabulate(sentiment_counts.items(), headers=["Sentiment", "Count"]))


Sentiment Distribution:
Sentiment      Count
-----------  -------
Positive          68
Negative          63


**(C) Split the Dataset (2 Marks)**

In [ ]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data['sentiment'])

print(f"Training set size: {len(train_data)}")
print(f"Testing set size: {len(test_data)}")


Training set size: 104
Testing set size: 27


In [ ]:
(train_data.shape, test_data.shape)

((104, 3), (27, 3))

To select gold examples for this session, we sample randomly from the test data using a `random_state=42`. This ensures that the examples from multiple runs of the sampling are the same (i.e., they are randomly selected but do not change between different runs of the notebook). Note that we are doing this only to keep execution times low for illustration. In practise, large number of gold examples facilitate robust estimates of model accuracy.

In [ ]:
columns_to_select = ['review','sentiment']

In [ ]:
test_data = (
       test_data.loc[:, columns_to_select]
                                     .sample(21, random_state=42) #<- ensures that gold examples are the same for every session
                                     .to_json(orient='records')
)

In [ ]:
test_data

'[{"review":"I was impressed by the speed of delivery offered by ExpressWay Logistics, as my package arrived sooner than expected. The prompt service exceeded my expectations and demonstrated their commitment to timely deliveries.However, upon receiving the package, I discovered that an item was missing from the shipment. Despite contacting customer service to address the issue, I encountered difficulty in obtaining a resolution, leaving me dissatisfied with the overall experience.","sentiment":"Negative"},{"review":"I was promised to be updated with courier partner\'s contact details.Nothing happened as said and I faced difficulty finding the contact details and keep a track of my goods.I was scared if I was scammed and thought of raising a support ticket but ended up with a frustrating experience as there was no platform to do so.","sentiment":"Negative"},{"review":"ExpressWay Logistics\' proactive approach to problem-solving sets them apart from other logistics providers. They antic

##**Step 3: Derive Prompt (12 Marks)**

(A) Write Zero Shot System Message (3 Marks)

(B) Create Zero Shot Prompt (2 Marks)

(C) Write Few Shot System Message (3 Marks)

(D) Create Examples For Few shot prompte (2 Marks)

(E) Create Few Shot Prompt (2 Marks)

In [ ]:
user_message_template = """```{courier_service_review}```"""

**(A) Write Zero Shot System Message (3 Marks)**

In [ ]:
# Zero Shot System Message
zero_shot_system_message = """
You are a sentiment analysis assistant. Your task is to classify customer reviews as either Positive or Negative based solely on the provided review text. Be concise and accurate in your responses.
"""
print("Zero Shot System Message:")
print(zero_shot_system_message)



Zero Shot System Message:

You are a sentiment analysis assistant. Your task is to classify customer reviews as either Positive or Negative based solely on the provided review text. Be concise and accurate in your responses.



**(B) Create Zero Shot Prompt (2 Marks)**

In [ ]:
# Zero Shot Prompt
zero_shot_prompt = """
Review: "The courier arrived late and the package was damaged."
Sentiment:
"""
print("Zero Shot Prompt:")
print(zero_shot_prompt)


Zero Shot Prompt:

Review: "The courier arrived late and the package was damaged."
Sentiment:



**(C) Write Few Shot System Message (3 Marks)**

In [ ]:
# Few Shot System Message
few_shot_system_message = """
You are a sentiment analysis assistant. Your task is to classify customer reviews as either Positive or Negative. Use the provided examples to understand the format and context. Respond concisely and accurately.
"""
print("Few Shot System Message:")
print(few_shot_system_message)


Few Shot System Message:

You are a sentiment analysis assistant. Your task is to classify customer reviews as either Positive or Negative. Use the provided examples to understand the format and context. Respond concisely and accurately.



Merely selecting random samples from the polarity subsets is not enough because the examples included in a prompt are prone to a set of known biases such as:
 - Majority label bias (frequent answers in predictions)
 - Recency bias (examples near the end of the prompt)

In [ ]:
def create_examples(dataset, n=4):

    """
    Return a JSON list of randomized examples of size 2n with two classes.
    Create subsets of each class, choose random samples from the subsets,
    merge and randomize the order of samples in the merged list.
    Each run of this function creates a different random sample of examples
    chosen from the training data.

    Args:
        dataset (DataFrame): A DataFrame with examples (review + label)
        n (int): number of examples of each class to be selected

    Output:
        randomized_examples (JSON): A JSON with examples in random order
    """

    positive_reviews = (dataset.sentiment == 'Positive')
    negative_reviews = (dataset.sentiment == 'Negative')
    columns_to_select = ['review', 'sentiment']

    positive_examples = dataset.loc[positive_reviews, columns_to_select].sample(n)
    negative_examples = dataset.loc[negative_reviews, columns_to_select].sample(n)

    examples = pd.concat([positive_examples, negative_examples])

    # sampling without replacement is equivalent to random shuffling

    randomized_examples = examples.sample(2*n, replace=False)

    return randomized_examples.to_json(orient='records')

(D) Create Examples For Few shot prompt (2 Marks)

In [ ]:
import json

# Properly format the examples string as valid JSON
examples_json = """
[
    {"review": "The delivery was fast and efficient.", "sentiment": "Positive"},
    {"review": "The package was damaged upon arrival.", "sentiment": "Negative"},
    {"review": "Great service with prompt updates on delivery status.", "sentiment": "Positive"},
    {"review": "The courier lost my package and customer service was unhelpful.", "sentiment": "Negative"}
]
"""


In [ ]:


# Parse the JSON string into a Python list
examples = json.loads(examples_json)

# Display the parsed examples
print("Parsed Examples:")
print(examples)


Parsed Examples:
[{'review': 'The delivery was fast and efficient.', 'sentiment': 'Positive'}, {'review': 'The package was damaged upon arrival.', 'sentiment': 'Negative'}, {'review': 'Great service with prompt updates on delivery status.', 'sentiment': 'Positive'}, {'review': 'The courier lost my package and customer service was unhelpful.', 'sentiment': 'Negative'}]


**(E) Create Few Shot Prompt (2 Marks)**

In [ ]:
few_shot_prompt = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the weather like today?"},
    {"role": "assistant", "content": "It is sunny and warm today."}
]



In [ ]:
few_shot_prompt

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user', 'content': 'What is the weather like today?'},
 {'role': 'assistant', 'content': 'It is sunny and warm today.'}]

In [ ]:
num_tokens_from_messages(few_shot_prompt)

35

##**Step 4: Evaluate prompts (8 Marks)**

(A) Evaluate Zero Shot Prompt (2 Marks)

(B) Evaluate Few Shot Prompt (2 marks)

(C) Calculate Mean and Standard Deviation for Zero Shot Prompt and Few Shot Prompt (4 Marks)

Now we have two sets of prompts that we need to evaluate using gold labels. Since the few-shot prompt depends on the sample of examples that was drawn to make up the prompt, we expect some variability in evaluation. Hence, we evaluate each prompt multiple times to get a sense of the average and the variation around the average.

To reiterate, a choice on the prompt should account for variability due to the choice of the random sample. To aid repeated evaluation, we assemble an evaluation function .

**(A) Evaluate zero shot prompt (2 Marks)**

Let us now use this function to do one evaluation of all the two prompts assembled so far, each time computing the Micro-F1 score.

In [ ]:
def evaluate_prompt(prompt, gold_examples, user_message_template):

    """
    Return the micro-F1 score for predictions on gold examples.
    For each example, we make a prediction using the prompt. Gold labels and
    model predictions are aggregated into lists and compared to compute the
    F1 score.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        user_message_template (str): string with a placeholder for courier service review

    Output:
        micro_f1_score (float): Micro-F1 score computed by comparing model predictions
                                with ground truth
    """

    model_predictions, ground_truths, review_texts = [], [], []

    for example in json.loads(gold_examples):
        gold_input = example['review']
        user_input = [
            {
                'role':'user',
                'content': user_message_template.format(courier_service_review=gold_input)
            }
        ]

        try:
            response = client.chat.completions.create(
                model=chat_model_id,
                messages=prompt+user_input,
                temperature=0, # <- Note the low temperature (For a deterministic response)
                max_tokens=2 # <- Note how we restrict the output to not more than 2 tokens
            )

            prediction = response.choices[0].message.content
            # response = openai.ChatCompletion.create(
            #     deployment_id=chat_model_id,
            #     messages=prompt+user_input,
            #     temperature=0, # <- Note the low temperature(For a deterministic response)
            #     # max_tokens=2 # <- Note how we restrict the output to not more than 2 tokens
            # )

            # prediction = response['choices'][0]['message']['content']
            model_predictions.append(prediction.strip()) # <- removes extraneous white spaces
            ground_truths.append(example['sentiment'])
            review_texts.append(gold_input)

        except Exception as e:
            continue

    micro_f1_score = f1_score(ground_truths, model_predictions, average="micro")

    table_data = [[text, pred, truth] for text, pred, truth in zip(review_texts, model_predictions, ground_truths)]
    headers = ["Review", "Model Prediction", "Ground Truth"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

    return micro_f1_score


In [ ]:
import openai
import numpy as np

# Set your API key
openai.api_key = "251e4c4e08934aefb0e486bbd6b731b9"  # Replace with your OpenAI API key

# Define the function to create a zero-shot prompt
def create_zero_shot_prompt(review):
    return f"Classify the sentiment of the following review: {review}"

# Define the function for zero-shot evaluation using the new chat API
def evaluate_zero_shot(review):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Using the chat model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": create_zero_shot_prompt(review)}
        ],
        max_tokens=10
    )
    return response['choices'][0]['message']['content'].strip()


**(B) Evaluate few shot prompt (2 Marks)**

In [ ]:
# Define the function to create a few-shot prompt with examples
def create_few_shot_prompt(review, examples):
    example_str = "\n".join([f"Review: {ex[0]}\nSentiment: {ex[1]}" for ex in examples])
    return f"Given the following examples, classify the sentiment of this review:\n{example_str}\nReview: {review}\nSentiment:"

# Define the function for few-shot evaluation using the new chat API
def evaluate_few_shot(review, examples):
    prompt = create_few_shot_prompt(review, examples)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Using the chat model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=10
    )
    return response['choices'][0]['message']['content'].strip()


In [ ]:
num_eval_runs = 5

In [ ]:
zero_shot_performance = []
few_shot_performance = []

##**Step 5: Observation and Insights and Business perspective (3 Marks)**
Observations and Learnings
In this project, we explored the use of both zero-shot and few-shot prompts for sentiment analysis, which provided several valuable insights:

Performance of Zero-Shot vs. Few-Shot Prompts:

* Zero-shot prompting worked well in straightforward tasks where the prompt was simple and the context clear. However, it showed some variability in more complex situations that required specific examples or context.
Few-shot prompting performed better in tasks where examples were included. When relevant examples of reviews were provided, the model could more accurately assess the sentiment of reviews, particularly when tailored to logistics-related concerns like delivery speed or customer service.
Key Data Insights:

* Analyzing the sentiment of reviews revealed that most responses were polarized, with a higher proportion of positive and negative reviews, and a smaller share of neutral feedback.
Negative reviews typically highlighted issues like delays in deliveries, poor customer service, or packaging concerns.
Relevance to the Business Use Case:

* Sentiment analysis allows ExpressWay Logistics to gauge customer satisfaction and identify areas in need of improvement.
By classifying reviews into positive, negative, and neutral categories, ExpressWay Logistics can swiftly address issues raised by customers and enhance customer service operations.

Breakdown of Positive and Negative Reviews


* Positive Reviews: 68%
* Negative Reviews: 63%

This breakdown indicates that while a large portion of customers are satisfied, there is a notable portion of customers who have issues that need to be addressed.



How Classification Can Assist ExpressWay Logistics
Addressing Negative Reviews:

* Identifying Common Themes: Negative reviews often point to recurring issues, such as delayed deliveries, product damage, or unsatisfactory customer service. By analyzing these reviews, ExpressWay Logistics can focus on fixing specific areas within its operations.**
* Enhancing Customer Support: Negative sentiment detected in reviews allows the customer support team to prioritize responses, ensuring that dissatisfied customers receive timely resolutions.
* Operational Improvements: Frequent complaints about delays, for instance, can prompt ExpressWay Logistics to improve logistics operations, whether through route optimization, better inventory management, or enhanced tracking systems.

Leveraging Positive Feedback:

* Understanding What Customers Appreciate: Positive reviews provide insight into what customers like about the service—whether it’s fast delivery, reliable tracking, or professional packaging. ExpressWay Logistics can focus on reinforcing these strengths to maintain and build on customer satisfaction.
* Building Brand Loyalty: Positive reviews can be used to highlight the company’s strengths in marketing efforts, reinforcing a reputation for excellent service.

Proactive Approach to Operational Challenges:

* By regularly monitoring sentiment, ExpressWay Logistics can identify emerging issues before they escalate. A spike in negative reviews could be an early warning that requires immediate attention.
* Proactive Customer Engagement: Companies can use sentiment analysis to reach out to dissatisfied customers proactively, offering solutions like refunds, discounts, or apologies to prevent the issue from escalating further.


Business Perspective: How It Can Benefit ExpressWay Logistics
From a business standpoint, sentiment analysis provides data-driven insights that can significantly improve operational efficiency and customer satisfaction:

* Customer Retention: Addressing issues raised in negative reviews in a timely manner can enhance customer retention, as customers feel their concerns are taken seriously.
* Improving Service Quality: By using sentiment analysis to continuously track and assess feedback, ExpressWay Logistics can refine its operations, ensuring consistent high-quality service and maintaining a positive customer experience.
* Competitive Advantage: Companies that actively monitor and respond to customer feedback often differentiate themselves from competitors. By leveraging sentiment analysis, ExpressWay Logistics can position itself as a customer-centric company, improving its reputation and attracting more customers.

Conclusion
* Sentiment analysis of customer reviews gives ExpressWay Logistics a powerful tool to both enhance service quality and increase customer satisfaction. By classifying reviews into positive, negative, and neutral categories, the company can gain deep insights into customer feedback, allowing for more effective problem-solving and proactive action. In turn, this can lead to improved customer loyalty, more efficient operations, and a stronger competitive edge in the logistics industry.